# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-d17b0d6009-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Politics Workflow Series ("Politicians in E.U.") 

Consider the following exploratory information need:

> You investigating the careers of politicians in the E.U.

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P17`     | country       | predicate |
| `wdt:P27`     | citizenship   | predicate |
| `wdt:P39`     | position held   | predicate |
| `wdt:P106`    | profession   | predicate |
| `wd:Q82955`   | politician | node      |
| `wd:Q46`      | Europe        | node |
| `wd:Q38`      | Italy          | node |



Also consider

```
?p wdt:P106/wdt:P279* wd:Q82955  . 
```

is the BGP to retrieve all **politicians**

## Workload Goals


1. Identify the BGP to retrieve E.U. countries and their politicians

2. Identify the BGP for obtaining other occupations and properties of politicians

3. How many politicians are recorder for each E.U. country?

4. Are there politicians with double citizenship?

5. Analyze the number of politicians in each country by occupation, for instance
 
   5.1 What are the top-3 occupations for a politician in Italy and France?
   
   5.2 What if you consider only politicians for which we don't have a date of death?
   
   5.3 Which politicians had a spouse that was also a politician? How many in each country?


In [1]:
# start your workflow here

In [2]:
queryString = """
SELECT COUNT(*)
WHERE { 

?p wdt:P106/wdt:P279 wd:Q82955  . 
} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '15556')]


1

I start by finding the graph pattern connecting a country and the EU

In [3]:
#Q1
#try to find the property to connect country and EU
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 

?pol wdt:P106/wdt:P279 wd:Q82955;
     wdt:P27 ?country.
?country ?p wd:Q46.

#Labels
?p <http://schema.org/name> ?pname .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P30'), ('pname', 'continent')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pname', 'part of')]


2

In [4]:
#Q2
#try to find the property to connect country and EU
queryString = """
SELECT DISTINCT ?o ?oname
WHERE { 

?pol wdt:P106/wdt:P279 wd:Q82955;
     wdt:P27 ?country.
?country wdt:P361 ?o.

#Labels
?o <http://schema.org/name> ?oname .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q9721810'), ('oname', 'Northern dynasties')]
[('o', 'http://www.wikidata.org/entity/Q756617'), ('oname', 'Danish Realm')]
[('o', 'http://www.wikidata.org/entity/Q25558247'), ('oname', 'Chinese civilization')]
[('o', 'http://www.wikidata.org/entity/Q83201'), ('oname', 'Non-Aligned Movement')]
[('o', 'http://www.wikidata.org/entity/Q43467'), ('oname', 'Indochina')]
[('o', 'http://www.wikidata.org/entity/Q263686'), ('oname', 'Southeast Europe')]
[('o', 'http://www.wikidata.org/entity/Q20716241'), ('oname', 'Ten Kingdoms')]
[('o', 'http://www.wikidata.org/entity/Q1065'), ('oname', 'United Nations')]
[('o', 'http://www.wikidata.org/entity/Q11596055'), ('oname', 'Qin Han')]
[('o', 'http://www.wikidata.org/entity/Q12557'), ('oname', 'Mongol Empire')]
[('o', 'http://www.wikidata.org/entity/Q12585'), ('oname', 'Latin America')]
[('o', 'http://www.wikidata.org/entity/Q13116'), ('oname', 'Benelux')]
[('o', 'http://www.wikidata.org/entity/Q15180'), ('

100

In [5]:
#Q3
#try to find the property to connect country and EU
queryString = """
SELECT DISTINCT ?o ?oname
WHERE { 

?pol wdt:P106/wdt:P279 wd:Q82955;
     wdt:P27 ?country.
?country wdt:P361 ?o;
         wdt:P30 wd:Q46.

#Labels
?o <http://schema.org/name> ?oname .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q756617'), ('oname', 'Danish Realm')]
[('o', 'http://www.wikidata.org/entity/Q263686'), ('oname', 'Southeast Europe')]
[('o', 'http://www.wikidata.org/entity/Q1065'), ('oname', 'United Nations')]
[('o', 'http://www.wikidata.org/entity/Q13116'), ('oname', 'Benelux')]
[('o', 'http://www.wikidata.org/entity/Q172107'), ('oname', 'Polish–Lithuanian Commonwealth')]
[('o', 'http://www.wikidata.org/entity/Q184'), ('oname', 'Belarus')]
[('o', 'http://www.wikidata.org/entity/Q2'), ('oname', 'Earth')]
[('o', 'http://www.wikidata.org/entity/Q21195'), ('oname', 'Scandinavia')]
[('o', 'http://www.wikidata.org/entity/Q215669'), ('oname', 'Allies of the First World War')]
[('o', 'http://www.wikidata.org/entity/Q23522'), ('oname', 'Balkans')]
[('o', 'http://www.wikidata.org/entity/Q27275'), ('oname', 'Central Asia')]
[('o', 'http://www.wikidata.org/entity/Q27293'), ('oname', 'Western Asia')]
[('o', 'http://www.wikidata.org/entity/Q27468'), ('oname', 'Easte

46

I found out that entity Q458 is EU, now i will check the path P361( part of) + Q458

In [6]:
#Q4
# ?s wdt:P361 wd:Q458 BGP for country belonging to EU
#count countries in EU
queryString = """
SELECT COUNT(?country)
WHERE { 

?country wdt:P361 wd:Q458.

} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '19')]


1

In [7]:
#Q5
#count how many politicians have each country in the EU
queryString = """
SELECT ?country (COUNT(?pol) AS ?howMany)
WHERE { 

?pol wdt:P106/wdt:P279 wd:Q82955;
     wdt:P27 ?country.
?country wdt:P361 wd:Q458.

}
GROUP BY ?country
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q145'), ('howMany', '47')]
[('country', 'http://www.wikidata.org/entity/Q31'), ('howMany', '7')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('howMany', '2602')]


3

In [8]:
#Q6
#as previous one but with Europe instead EU
queryString = """
SELECT ?country (COUNT(DISTINCT ?pol) AS ?howMany)
WHERE { 

?pol wdt:P106/wdt:P279 wd:Q82955;
     wdt:P27 ?country.
?country wdt:P30 wd:Q46.

}
GROUP BY ?country
ORDER BY DESC(?howMany)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q29'), ('howMany', '2600')]
[('country', 'http://www.wikidata.org/entity/Q15180'), ('howMany', '2091')]
[('country', 'http://www.wikidata.org/entity/Q1747689'), ('howMany', '1465')]
[('country', 'http://www.wikidata.org/entity/Q159'), ('howMany', '642')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('howMany', '493')]
[('country', 'http://www.wikidata.org/entity/Q212'), ('howMany', '264')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('howMany', '185')]
[('country', 'http://www.wikidata.org/entity/Q28'), ('howMany', '169')]
[('country', 'http://www.wikidata.org/entity/Q133356'), ('howMany', '114')]
[('country', 'http://www.wikidata.org/entity/Q139319'), ('howMany', '104')]
[('country', 'http://www.wikidata.org/entity/Q243610'), ('howMany', '91')]
[('country', 'http://www.wikidata.org/entity/Q34'), ('howMany', '76')]
[('country', 'http://www.wikidata.org/entity/Q184'), ('howMany', '73')]
[('country', 'http://www.wikida

100

In [9]:
#Q7
#using Europe instead of EU the results are very different, check why
queryString = """
SELECT DISTINCT ?country ?cname
WHERE { 

?pol wdt:P106/wdt:P279 wd:Q82955;
     wdt:P27 ?country.
?country wdt:P30 wd:Q46.
FILTER NOT EXISTS{?country wdt:P361 wd:Q458}

#Labels
?country <http://schema.org/name> ?cname .

}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q139319'), ('cname', 'Russian Republic')]
[('country', 'http://www.wikidata.org/entity/Q1508143'), ('cname', 'Ukrainian State')]
[('country', 'http://www.wikidata.org/entity/Q174306'), ('cname', 'Republic of Genova')]
[('country', 'http://www.wikidata.org/entity/Q706018'), ('cname', 'Electorate of Brunswick-Lüneburg')]
[('country', 'http://www.wikidata.org/entity/Q10957559'), ('cname', 'Principality of Moldavia')]
[('country', 'http://www.wikidata.org/entity/Q146246'), ('cname', 'Francia')]
[('country', 'http://www.wikidata.org/entity/Q171348'), ('cname', 'Crown of the Kingdom of Poland')]
[('country', 'http://www.wikidata.org/entity/Q583038'), ('cname', 'Ostrogothic Kingdom')]
[('country', 'http://www.wikidata.org/entity/Q1140152'), ('cname', 'Irish Republic')]
[('country', 'http://www.wikidata.org/entity/Q243652'), ('cname', 'Hungarian Soviet Republic')]
[('country', 'http://www.wikidata.org/entity/Q684030'), ('cname', 'Principalit

100

The previous path to connect a country to UE doesn't seem right because it retrieves only 19 countries and as a consequence some ones, like Germany,
are not present. 
I'll try the reverse path to see if there are some other paths.

In [10]:
#Q8
#Germany is not listed as member of EU, check the reverse path
queryString = """
SELECT DISTINCT ?o ?oname
WHERE { 

wd:Q458 ?p ?o.

#Labels
?o <http://schema.org/name> ?oname .

}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q3323440'), ('oname', 'Princess of Asturias Award for Concord')]
[('o', 'http://www.wikidata.org/entity/Q10858397'), ('oname', 'WikiProject European Union')]
[('o', 'http://www.wikidata.org/entity/Q11146'), ('oname', 'Maastricht Treaty')]
[('o', 'http://www.wikidata.org/entity/Q223'), ('oname', 'Greenland')]
[('o', 'http://www.wikidata.org/entity/Q23681'), ('oname', 'Turkish Republic of Northern Cyprus')]
[('o', 'http://www.wikidata.org/entity/Q8889'), ('oname', 'European Parliament')]
[('o', 'http://www.wikidata.org/entity/Q9072'), ('oname', 'Estonian')]
[('o', 'http://www.wikidata.org/entity/Q9078'), ('oname', 'Latvian')]
[('o', 'http://www.wikidata.org/entity/Q9083'), ('oname', 'Lithuanian')]
[('o', 'http://www.wikidata.org/entity/Q7987828'), ('oname', 'Western European Summer Time')]
[('o', 'http://www.wikidata.org/entity/Q480222'), ('oname', 'Publications Office of the European Union')]
[('o', 'http://www.wikidata.org/entity/Q3249016'

100

In [11]:
#Q9
#Germany is not listed as member of EU, check the reverse path
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 

wd:Q458 ?p wd:Q183.

#Labels
?p <http://schema.org/name> ?pname .

}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P150'), ('pname', 'contains administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part')]


2

The BGP wd:Q458 wdt:P527 ?country seems the right one to retrieve all the members of EU. Let's count the countries retrieved to check if it is true.

In [12]:
#Q10
#count member of EU by using P527
queryString = """
SELECT COUNT(?country)
WHERE { 

wd:Q458 wdt:P527 ?country.

}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '27')]


1

Run the query to count the number of politicians for each EU country

In [13]:
#Q11
#run Q6 using the reverse path (from EU to country)
queryString = """
SELECT ?country (COUNT(DISTINCT ?pol) AS ?howMany)
WHERE { 

?pol wdt:P106/wdt:P279 wd:Q82955;
     wdt:P27 ?country.
wd:Q458 wdt:P527 ?country.

}
GROUP BY ?country
ORDER BY DESC(?howMany)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q29'), ('howMany', '2600')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('howMany', '493')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('howMany', '185')]
[('country', 'http://www.wikidata.org/entity/Q28'), ('howMany', '169')]
[('country', 'http://www.wikidata.org/entity/Q34'), ('howMany', '76')]
[('country', 'http://www.wikidata.org/entity/Q35'), ('howMany', '68')]
[('country', 'http://www.wikidata.org/entity/Q36'), ('howMany', '40')]
[('country', 'http://www.wikidata.org/entity/Q40'), ('howMany', '38')]
[('country', 'http://www.wikidata.org/entity/Q215'), ('howMany', '29')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('howMany', '19')]
[('country', 'http://www.wikidata.org/entity/Q33'), ('howMany', '13')]
[('country', 'http://www.wikidata.org/entity/Q41'), ('howMany', '11')]
[('country', 'http://www.wikidata.org/entity/Q218'), ('howMany', '9')]
[('country', 'http://www.wikidata.org/entity/Q32'), ('howMany

24

In [14]:
#Q12
#retrieve italian politicians
queryString = """
SELECT ?pol ?name
WHERE { 

?pol wdt:P106/wdt:P279 wd:Q82955;
     wdt:P27 wd:Q38.
#Labels
?pol <http://schema.org/name> ?name .

}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('pol', 'http://www.wikidata.org/entity/Q6554677'), ('name', 'Lino Guzzella')]
[('pol', 'http://www.wikidata.org/entity/Q104867229'), ('name', 'Roberto Sgalla')]
[('pol', 'http://www.wikidata.org/entity/Q3617921'), ('name', 'Annarita Buttafuoco')]
[('pol', 'http://www.wikidata.org/entity/Q21101292'), ('name', 'Mario Beccaria')]
[('pol', 'http://www.wikidata.org/entity/Q92911939'), ('name', 'Carlo Emanuele Filiberto Giacinto di Simiana')]
[('pol', 'http://www.wikidata.org/entity/Q97022278'), ('name', 'Silvano Capraro')]
[('pol', 'http://www.wikidata.org/entity/Q91449519'), ('name', 'Rino Clerici')]
[('pol', 'http://www.wikidata.org/entity/Q3621422'), ('name', 'Arcangelo Pezzella')]
[('pol', 'http://www.wikidata.org/entity/Q107271734'), ('name', 'Giovanni Lapi')]
[('pol', 'http://www.wikidata.org/entity/Q18418864'), ('name', 'Libero Mazza')]
[('pol', 'http://www.wikidata.org/entity/Q105979780'), ('name', 'Carlo Giacomo Fanchini')]
[('pol', 'http://www.wikidata.org/entity/Q107344

17

I got very few results from the previous one, i'll add the iteration over P279 in the next one 

In [2]:
#Q13
#few results, iterate over P279
#count italian politicians
queryString = """
SELECT COUNT(?pol)
WHERE { 

?pol wdt:P106/wdt:P279* wd:Q82955;
     wdt:P27 wd:Q38.
#Labels
?pol <http://schema.org/name> ?name .

}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '12131')]


1

In [3]:
#TASK 1 ENDS HERE: BGP to retrieve an EU country and its politicians
queryString = """
SELECT ?country ?pol
WHERE { 

?pol wdt:P106/wdt:P279* wd:Q82955;
     wdt:P27 ?country.
wd:Q458 wdt:P527 ?country. #EU
}
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q142'), ('pol', 'http://www.wikidata.org/entity/Q49164475')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('pol', 'http://www.wikidata.org/entity/Q21492547')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('pol', 'http://www.wikidata.org/entity/Q20895513')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('pol', 'http://www.wikidata.org/entity/Q99654425')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('pol', 'http://www.wikidata.org/entity/Q10843521')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('pol', 'http://www.wikidata.org/entity/Q21427157')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('pol', 'http://www.wikidata.org/entity/Q33113557')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('pol', 'http://www.wikidata.org/entity/Q2833524')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('pol', 'http://www.wikidata.org/entity/Q50400574')]
[('country', 'http://www.wikidata.org/e

10

<h3> Task 1 ENDS HERE </h3>
In the first task I found out how to check if a country is a member of EU, I counted the number of politicians for each country in EU. Mixing these 
two results i found out the BGP to retrieve all the politicians of one EU country and tested it for Italy.

<h3> Task 2 STARTS HERE </h3>
Identify the BGP for obtaining other occupations and properties of politicians

In [3]:
#Q14
#retrieve some predicates of italian politicians
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 

?pol wdt:P106/wdt:P279* wd:Q82955;
     wdt:P27 wd:Q38;
     ?p ?o.
FILTER(!isLiteral(?o))
#Labels
?p <http://schema.org/name> ?pname .

}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P102'), ('pname', 'member of political party')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('pname', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('pname', 'place of birth')]
[('p', 'http://www.wikidata.org/prop/direct/P20'), ('pname', 'place of death')]
[('p', 'http://www.wikidata.org/prop/direct/P21'), ('pname', 'sex or gender')]
[('p', 'http://www.wikidata.org/prop/direct/P27'), ('pname', 'country of citizenship')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P463'), ('pname', 'member of')]
[('p', 'http://www.wikidata.org/prop/direct/P6886'), ('pname', 'writing language')]
[('p', 'http://www.wikidata.org/prop/direct/P734'), ('pname', 'family name')]
[('p', 'http://www.wikidata.org/prop/direct/P735'), ('pname', 'given name')]
[

100

I found out some interesting properties such as occupation (P106), member of political party (P102), political ideology (P1142), employer (P108), 
field of work (P101)

In [4]:
#Q15
#interesting properties: occupation (P106), member of political party (P102), political ideology (P1142), employer (P108), field of work (P101)
queryString = """
SELECT COUNT(DISTINCT ?pol)
WHERE { 

?pol wdt:P106/wdt:P279* wd:Q82955;
     wdt:P27 wd:Q38;
     wdt:P106 ?occ.
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '12079')]


1

Checking the other occupations by excluding politics in filter clause. I compute the query only on italian politicians to test.

In [6]:
#Q16
#exclude from result set the ones related to politics
queryString = """
SELECT ?occ ?name ?howMany WHERE {
{SELECT ?occ (COUNT(?pol) AS ?howMany)
WHERE { 

?pol wdt:P106/wdt:P279* wd:Q82955;
     wdt:P27 wd:Q38;
     wdt:P106 ?occ.
FILTER(?occ!=wd:Q82955)
}
GROUP BY ?occ
}
#Labels
?occ <http://schema.org/name> ?name .
}
ORDER BY DESC(?howMany)
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('occ', 'http://www.wikidata.org/entity/Q1930187'), ('name', 'journalist'), ('howMany', '733')]
[('occ', 'http://www.wikidata.org/entity/Q40348'), ('name', 'lawyer'), ('howMany', '715')]
[('occ', 'http://www.wikidata.org/entity/Q36180'), ('name', 'writer'), ('howMany', '347')]
[('occ', 'http://www.wikidata.org/entity/Q37226'), ('name', 'teacher'), ('howMany', '235')]
[('occ', 'http://www.wikidata.org/entity/Q131524'), ('name', 'entrepreneur'), ('howMany', '227')]
[('occ', 'http://www.wikidata.org/entity/Q1622272'), ('name', 'university teacher'), ('howMany', '225')]
[('occ', 'http://www.wikidata.org/entity/Q15627169'), ('name', 'trade unionist'), ('howMany', '190')]
[('occ', 'http://www.wikidata.org/entity/Q188094'), ('name', 'economist'), ('howMany', '181')]
[('occ', 'http://www.wikidata.org/entity/Q39631'), ('name', 'physician'), ('howMany', '173')]
[('occ', 'http://www.wikidata.org/entity/Q81096'), ('name', 'engineer'), ('howMany', '151')]
[('occ', 'http://www.wikidata.org/

25

Extend the result set on all the EU countries

In [7]:
#Q17
#as before but consider all countries in EU
queryString = """
SELECT ?occ ?name ?howMany WHERE {
{SELECT ?occ (COUNT(?pol) AS ?howMany)
WHERE { 

?pol wdt:P106/wdt:P279* wd:Q82955;
     wdt:P27 ?country;
     wdt:P106 ?occ.
wd:Q458 wdt:P527 ?country. #EU
FILTER(?occ!=wd:Q82955)
}
GROUP BY ?occ
}
#Labels
?occ <http://schema.org/name> ?name .
}
ORDER BY DESC(?howMany)
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('occ', 'http://www.wikidata.org/entity/Q1749879'), ('name', 'pensioner'), ('howMany', '14827')]
[('occ', 'http://www.wikidata.org/entity/Q40348'), ('name', 'lawyer'), ('howMany', '9502')]
[('occ', 'http://www.wikidata.org/entity/Q1930187'), ('name', 'journalist'), ('howMany', '5868')]
[('occ', 'http://www.wikidata.org/entity/Q97768274'), ('name', 'anciens cadres'), ('howMany', '5440')]
[('occ', 'http://www.wikidata.org/entity/Q3062119'), ('name', 'farm operator'), ('howMany', '4908')]
[('occ', 'http://www.wikidata.org/entity/Q36180'), ('name', 'writer'), ('howMany', '4848')]
[('occ', 'http://www.wikidata.org/entity/Q185351'), ('name', 'jurist'), ('howMany', '4149')]
[('occ', 'http://www.wikidata.org/entity/Q1622272'), ('name', 'university teacher'), ('howMany', '3903')]
[('occ', 'http://www.wikidata.org/entity/Q47064'), ('name', 'military personnel'), ('howMany', '3308')]
[('occ', 'http://www.wikidata.org/entity/Q193391'), ('name', 'diplomat'), ('howMany', '3163')]
[('occ', '

25

I am curios about Political Ideology property (P1142), i am going to see some results on italian politicians.

In [8]:
#Q18
#most common political ideology (P1142) in italy
queryString = """
SELECT ?id (COUNT(?pol) AS ?howMany) WHERE{
?pol wdt:P106/wdt:P279* wd:Q82955;
     wdt:P27 wd:Q38;
     wdt:P1142 ?id.
}
GROUP BY ?id
ORDER BY DESC(?howMany)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('id', 'http://www.wikidata.org/entity/Q6186'), ('howMany', '3')]
[('id', 'http://www.wikidata.org/entity/Q6223'), ('howMany', '2')]
[('id', 'http://www.wikidata.org/entity/Q912036'), ('howMany', '2')]
[('id', 'http://www.wikidata.org/entity/Q7264'), ('howMany', '2')]
[('id', 'http://www.wikidata.org/entity/Q223200'), ('howMany', '1')]
[('id', 'http://www.wikidata.org/entity/Q7785048'), ('howMany', '1')]
[('id', 'http://www.wikidata.org/entity/Q7876201'), ('howMany', '1')]
[('id', 'http://www.wikidata.org/entity/Q194236'), ('howMany', '1')]
[('id', 'http://www.wikidata.org/entity/Q6216'), ('howMany', '1')]
[('id', 'http://www.wikidata.org/entity/Q210189'), ('howMany', '1')]


10

Since I got very few results, I'll try to use "Political Party" (P102).

In [10]:
#Q19
#check if there are more results using member of political party (P102)
queryString = """
SELECT ?id ?name ?howMany WHERE {
{SELECT ?id (COUNT(?pol) AS ?howMany) WHERE{
?pol wdt:P106/wdt:P279* wd:Q82955;
     wdt:P27 wd:Q38;
     wdt:P102 ?id.
}}
#Labels
?id <http://schema.org/name> ?name .
}
GROUP BY ?id
ORDER BY DESC(?howMany)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('id', 'http://www.wikidata.org/entity/Q815348'), ('name', 'Christian Democracy'), ('howMany', '1991')]
[('id', 'http://www.wikidata.org/entity/Q47729'), ('name', 'Democratic Party'), ('howMany', '1243')]
[('id', 'http://www.wikidata.org/entity/Q461886'), ('name', 'Italian Communist Party'), ('howMany', '1125')]
[('id', 'http://www.wikidata.org/entity/Q590750'), ('name', 'Italian Socialist Party'), ('howMany', '880')]
[('id', 'http://www.wikidata.org/entity/Q47750'), ('name', 'Lega Nord'), ('howMany', '520')]
[('id', 'http://www.wikidata.org/entity/Q47817'), ('name', 'Five Star Movement'), ('howMany', '442')]
[('id', 'http://www.wikidata.org/entity/Q139596'), ('name', 'National Fascist Party'), ('howMany', '379')]
[('id', 'http://www.wikidata.org/entity/Q215350'), ('name', 'Forza Italia'), ('howMany', '344')]
[('id', 'http://www.wikidata.org/entity/Q662849'), ('name', 'National Alliance'), ('howMany', '299')]
[('id', 'http://www.wikidata.org/entity/Q47720'), ('name', 'The Peop

10

Retrieve some predicates of the political parties to see if they are somehow connected to ideology.

In [12]:
#Q20
#check predicates of political party (P102)
queryString = """
SELECT DISTINCT ?p ?pname WHERE {
?pol wdt:P106/wdt:P279* wd:Q82955;
     wdt:P27 wd:Q38;
     wdt:P102 ?id.
?id ?p ?o.
FILTER(!isLiteral(?o))
#Labels
?p <http://schema.org/name> ?pname .
}
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1142'), ('pname', 'political ideology')]
[('p', 'http://www.wikidata.org/prop/direct/P1387'), ('pname', 'political alignment')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('pname', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('pname', 'headquarters location')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P1830'), ('pname', 'owner of')]
[('p', 'http://www.wikidata.org/prop/direct/P2184'), ('pname', 'history of topic')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P462'), ('pname', 'color')]
[('p', 'http://www.wikidata.org/prop/direct/P463'), ('pname', 'member of')]


10

Using the path from party to ideology, retrieve the most common politican ideologies in Italy.

In [13]:
#Q21
#Q18 using the new path discovered above (politician->party->ideology)
queryString = """
SELECT ?id ?name ?howMany WHERE{

{SELECT ?id (COUNT(?pol) AS ?howMany) WHERE{
?pol wdt:P106/wdt:P279* wd:Q82955;
     wdt:P27 wd:Q38;
     wdt:P102 ?party.
?party wdt:P1142 ?id.
     
}
GROUP BY ?id}
#Labels
?id <http://schema.org/name> ?name .
}
ORDER BY DESC(?howMany)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('id', 'http://www.wikidata.org/entity/Q121254'), ('name', 'social democracy'), ('howMany', '2881')]
[('id', 'http://www.wikidata.org/entity/Q6223'), ('name', 'fascism'), ('howMany', '2389')]
[('id', 'http://www.wikidata.org/entity/Q3781399'), ('name', 'pro-Europeanism'), ('howMany', '1465')]
[('id', 'http://www.wikidata.org/entity/Q206743'), ('name', 'Eurocommunism'), ('howMany', '1331')]
[('id', 'http://www.wikidata.org/entity/Q1080800'), ('name', 'Christian left'), ('howMany', '1263')]
[('id', 'http://www.wikidata.org/entity/Q1414298'), ('name', 'social conservatism'), ('howMany', '1227')]
[('id', 'http://www.wikidata.org/entity/Q6186'), ('name', 'communism'), ('howMany', '1210')]
[('id', 'http://www.wikidata.org/entity/Q7272'), ('name', 'socialism'), ('howMany', '1137')]
[('id', 'http://www.wikidata.org/entity/Q209158'), ('name', 'Christian democracy'), ('howMany', '984')]
[('id', 'http://www.wikidata.org/entity/Q1043096'), ('name', 'liberal conservatism'), ('howMany', '91

10

Retrieve the most common political ideologies in EU, grouped by country.

In [15]:
#Q22
#most common political ideology in EU (per country)
queryString = """
SELECT ?id ?name ?country ?cname ?howMany WHERE{

{SELECT ?id ?country (COUNT(?pol) AS ?howMany) WHERE{
?pol wdt:P106/wdt:P279* wd:Q82955;
     wdt:P27 ?country;
     wdt:P102 ?party.
?party wdt:P1142 ?id.
wd:Q458 wdt:P527 ?country. #EU
}
GROUP BY ?id ?country}
#Labels
?id <http://schema.org/name> ?name .
?country <http://schema.org/name> ?cname .
}
ORDER BY DESC(?howMany)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('id', 'http://www.wikidata.org/entity/Q209158'), ('name', 'Christian democracy'), ('country', 'http://www.wikidata.org/entity/Q183'), ('cname', 'Germany'), ('howMany', '9531')]
[('id', 'http://www.wikidata.org/entity/Q121254'), ('name', 'social democracy'), ('country', 'http://www.wikidata.org/entity/Q183'), ('cname', 'Germany'), ('howMany', '8521')]
[('id', 'http://www.wikidata.org/entity/Q7169'), ('name', 'conservatism'), ('country', 'http://www.wikidata.org/entity/Q183'), ('cname', 'Germany'), ('howMany', '7027')]
[('id', 'http://www.wikidata.org/entity/Q3781399'), ('name', 'pro-Europeanism'), ('country', 'http://www.wikidata.org/entity/Q183'), ('cname', 'Germany'), ('howMany', '6525')]
[('id', 'http://www.wikidata.org/entity/Q1043096'), ('name', 'liberal conservatism'), ('country', 'http://www.wikidata.org/entity/Q183'), ('cname', 'Germany'), ('howMany', '6432')]
[('id', 'http://www.wikidata.org/entity/Q3781399'), ('name', 'pro-Europeanism'), ('country', 'http://www.wikid

10

<h3> Task 2 ENDS HERE</h3>
I found out that most italian politicians are also writer and journalist or have a legal career. On top of that I explored other properties of the 
politicians and then I found out the connection between a person and his/her political ideology by going throught the political party he/she is member
of. To conclude I discovered that "Social Democracy" and "Christian Democracy" are/were the most common ideologies in Italy and Europe, respectively.

<h3> Task 3 STARTS HERE </h3>
Using Q11 with iteration over P279 we can get the number of politicians in each EU country

In [3]:
#Q23
#how many politicians for each EU country
queryString = """
SELECT ?name (COUNT(DISTINCT ?pol) AS ?howMany)
WHERE { 

?pol wdt:P106/wdt:P279* wd:Q82955;
     wdt:P27 ?country.
wd:Q458 wdt:P527 ?country.

#Labels
?country <http://schema.org/name> ?name .
}
GROUP BY ?name
ORDER BY DESC(?howMany)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('name', 'France'), ('howMany', '81679')]
[('name', 'Germany'), ('howMany', '35569')]
[('name', 'Spain'), ('howMany', '33257')]
[('name', 'Italy'), ('howMany', '12079')]
[('name', 'Romania'), ('howMany', '8770')]
[('name', 'Sweden'), ('howMany', '8338')]
[('name', 'Belgium'), ('howMany', '8090')]
[('name', 'Austria'), ('howMany', '7134')]
[('name', 'Poland'), ('howMany', '5264')]
[('name', 'Greece'), ('howMany', '4830')]
[('name', 'Hungary'), ('howMany', '4363')]
[('name', 'Finland'), ('howMany', '4215')]
[('name', 'Denmark'), ('howMany', '2883')]
[('name', 'Portugal'), ('howMany', '2326')]
[('name', 'Czech Republic'), ('howMany', '2257')]
[('name', 'Bulgaria'), ('howMany', '1545')]
[('name', 'Estonia'), ('howMany', '1436')]
[('name', 'Slovenia'), ('howMany', '1295')]
[('name', 'Lithuania'), ('howMany', '1208')]
[('name', 'Luxembourg'), ('howMany', '1118')]
[('name', 'Latvia'), ('howMany', '1111')]
[('name', 'Netherlands'), ('howMany', '849')]
[('name', 'Cyprus'), ('howMany', 

27

<h3> Task 4 STARTS HERE </h3>
Are there politicians with double citizenship?


From Q14 I know that P27 is the property for "Country of citizenship", check if politicians can have more than citizenships.

In [9]:
#Q24
#From Q14 I know that P27 is the property for "Country of citizenship"
#check how if politicians can have more than country
queryString = """
SELECT ?pol 
WHERE { 

?pol wdt:P106/wdt:P279* wd:Q82955;
     wdt:P27 ?c.
}
GROUP BY ?pol
HAVING( COUNT(?c) > 1)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('pol', 'http://www.wikidata.org/entity/Q3779978')]
[('pol', 'http://www.wikidata.org/entity/Q63323217')]
[('pol', 'http://www.wikidata.org/entity/Q106689762')]
[('pol', 'http://www.wikidata.org/entity/Q3770874')]
[('pol', 'http://www.wikidata.org/entity/Q16027100')]
[('pol', 'http://www.wikidata.org/entity/Q3182048')]
[('pol', 'http://www.wikidata.org/entity/Q19521750')]
[('pol', 'http://www.wikidata.org/entity/Q9357271')]
[('pol', 'http://www.wikidata.org/entity/Q17385988')]
[('pol', 'http://www.wikidata.org/entity/Q5721267')]


10

Number of politicians with multiple citizenships

In [10]:
#Q25
#check how many have multiple citizenships
queryString = """
SELECT (COUNT(DISTINCT ?pol) AS ?howMany){
SELECT ?pol 
WHERE { 

?pol wdt:P106/wdt:P279* wd:Q82955;
     wdt:P27 ?c.
}
GROUP BY ?pol
HAVING( COUNT(?c) > 1)
}
"""

print("Results")
run_query(queryString)

Results
[('howMany', '48558')]


1

Number of politicians with multiple citizenships but considering EU countries only.

In [11]:
#Q26
#check how many have multiple citizenships but considering EU countries only
queryString = """
SELECT (COUNT(DISTINCT ?pol) AS ?howMany){
SELECT ?pol 
WHERE { 

?pol wdt:P106/wdt:P279* wd:Q82955;
     wdt:P27 ?c.
wd:Q458 wdt:P527 ?c. #EU
}
GROUP BY ?pol
HAVING( COUNT(?c) > 1)
}
"""

print("Results")
run_query(queryString)

Results
[('howMany', '3307')]


1

Retrieve the politicians with the highest number of different citizenships.

In [16]:
#Q27
#politicians with the highest number of citizenships in EU countries
queryString = """
SELECT ?pol (COUNT(DISTINCT ?c) AS ?countries)
WHERE { 

?pol wdt:P106/wdt:P279* wd:Q82955;
     wdt:P27 ?c.
wd:Q458 wdt:P527 ?c. #EU
}
GROUP BY ?pol
ORDER BY DESC(?countries)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('pol', 'http://www.wikidata.org/entity/Q76343'), ('countries', '3')]
[('pol', 'http://www.wikidata.org/entity/Q12765455'), ('countries', '3')]
[('pol', 'http://www.wikidata.org/entity/Q1733194'), ('countries', '3')]
[('pol', 'http://www.wikidata.org/entity/Q114605'), ('countries', '3')]
[('pol', 'http://www.wikidata.org/entity/Q6244066'), ('countries', '3')]
[('pol', 'http://www.wikidata.org/entity/Q10994'), ('countries', '3')]
[('pol', 'http://www.wikidata.org/entity/Q365761'), ('countries', '3')]
[('pol', 'http://www.wikidata.org/entity/Q3051975'), ('countries', '3')]
[('pol', 'http://www.wikidata.org/entity/Q2492593'), ('countries', '3')]
[('pol', 'http://www.wikidata.org/entity/Q151209'), ('countries', '3')]


10

In [17]:
#Q28
#let's have a look at Q76343 just for curiosity
queryString = """
SELECT ?name (GROUP_CONCAT(?cname;SEPARATOR=",") AS ?countries)
WHERE { 

wd:Q76343 wdt:P27 ?c.
wd:Q458 wdt:P527 ?c. #EU

#Labels
?c <http://schema.org/name> ?cname .
wd:Q76343 <http://schema.org/name> ?name .

}
GROUP BY ?name
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('name', 'Otto von Habsburg'), ('countries', 'Germany,Croatia,Austria')]


1

As before, retrieve the politicians with the highest number of citizenships but considering the whole world.

In [18]:
#Q29
#as Q27 but considering the whole world
queryString = """
SELECT ?pol (COUNT(DISTINCT ?c) AS ?countries)
WHERE { 

?pol wdt:P106/wdt:P279* wd:Q82955;
     wdt:P27 ?c.
}
GROUP BY ?pol
ORDER BY DESC(?countries)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('pol', 'http://www.wikidata.org/entity/Q355648'), ('countries', '10')]
[('pol', 'http://www.wikidata.org/entity/Q30528417'), ('countries', '8')]
[('pol', 'http://www.wikidata.org/entity/Q768911'), ('countries', '8')]
[('pol', 'http://www.wikidata.org/entity/Q55196521'), ('countries', '7')]
[('pol', 'http://www.wikidata.org/entity/Q12150042'), ('countries', '7')]
[('pol', 'http://www.wikidata.org/entity/Q725435'), ('countries', '7')]
[('pol', 'http://www.wikidata.org/entity/Q76343'), ('countries', '7')]
[('pol', 'http://www.wikidata.org/entity/Q56259418'), ('countries', '7')]
[('pol', 'http://www.wikidata.org/entity/Q61052634'), ('countries', '7')]
[('pol', 'http://www.wikidata.org/entity/Q268003'), ('countries', '7')]


10

In [19]:
#Q30
#let's have a look at Q355648 just for curiosity
queryString = """
SELECT ?name (GROUP_CONCAT(?cname;SEPARATOR=",") AS ?countries)
WHERE { 

wd:Q355648 wdt:P27 ?c.

#Labels
?c <http://schema.org/name> ?cname .
wd:Q355648 <http://schema.org/name> ?name .

}
GROUP BY ?name
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('name', 'Adem Demaçi'), ('countries', "Kosovo,Serbia,Socialist Federal Republic of Yugoslavia,Kingdom of Yugoslavia,Serbia and Montenegro,Democratic Federal Yugoslavia,Federal Republic of Yugoslavia,Federal People's Republic of Yugoslavia,Albanian Kingdom,Albanian Kingdom")]


1

Retrieve politicians with the highest number of citizenships and at least one citizenship in EU.

In [20]:
#Q31
#politicians with the highest number of citizenships and at least one citizenship in EU
queryString = """
SELECT ?pol (COUNT(DISTINCT ?c) AS ?countries)
WHERE { 

?pol wdt:P27 ?c.
{
SELECT ?pol
WHERE { 

?pol wdt:P106/wdt:P279* wd:Q82955;
     wdt:P27 ?c.
wd:Q458 wdt:P527 ?c. #EU
}
}

}
GROUP BY ?pol
ORDER BY DESC(?countries)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('pol', 'http://www.wikidata.org/entity/Q78864'), ('countries', '7')]
[('pol', 'http://www.wikidata.org/entity/Q760748'), ('countries', '7')]
[('pol', 'http://www.wikidata.org/entity/Q76343'), ('countries', '7')]
[('pol', 'http://www.wikidata.org/entity/Q12294583'), ('countries', '7')]
[('pol', 'http://www.wikidata.org/entity/Q1986289'), ('countries', '6')]
[('pol', 'http://www.wikidata.org/entity/Q78741'), ('countries', '6')]
[('pol', 'http://www.wikidata.org/entity/Q29008'), ('countries', '6')]
[('pol', 'http://www.wikidata.org/entity/Q44517'), ('countries', '6')]
[('pol', 'http://www.wikidata.org/entity/Q6092246'), ('countries', '6')]
[('pol', 'http://www.wikidata.org/entity/Q112081'), ('countries', '6')]


10

<h3> Task 4 ENDS HERE </h3>
Using P27 I found out that about 3000 politicians have more than one citizenships in EU. The actual record is 3, while if we consider politician with
multiple citizenships all over the world the record is 10. However data seem to be a bit noisy if we consider countries outside EU because are kept in mind different variant of the same country (for example: Serbia and Serbia & Montenegro and Yugoslavia). 

<h3> Task 5 STARTS HERE </h3>
What are the top-3 occupations for a politician in Italy and France?

Retrieve the top 3 occupations for a politician in Italy

In [2]:
#Q32
#from Q8 I know that France is wd:Q142 while Italy is wd:Q38
#retrieve top 3 occupations in Italy
queryString = """
SELECT ?occ ?name ?howMany WHERE {
{SELECT ?occ (COUNT(?pol) AS ?howMany)
WHERE { 

?pol wdt:P106/wdt:P279* wd:Q82955;
     wdt:P27 wd:Q38;
     wdt:P106 ?occ.
FILTER(?occ!=wd:Q82955)
}
GROUP BY ?occ
}
#Labels
?occ <http://schema.org/name> ?name .
}
ORDER BY DESC(?howMany)
LIMIT 3
"""

print("Results")
run_query(queryString)

Results
[('occ', 'http://www.wikidata.org/entity/Q1930187'), ('name', 'journalist'), ('howMany', '733')]
[('occ', 'http://www.wikidata.org/entity/Q40348'), ('name', 'lawyer'), ('howMany', '715')]
[('occ', 'http://www.wikidata.org/entity/Q36180'), ('name', 'writer'), ('howMany', '347')]


3

Same for french politicians.

In [3]:
#Q33
#retrieve top 3 occupations in France
queryString = """
SELECT ?occ ?name ?howMany WHERE {
{SELECT ?occ (COUNT(?pol) AS ?howMany)
WHERE { 

?pol wdt:P106/wdt:P279* wd:Q82955;
     wdt:P27 wd:Q142;
     wdt:P106 ?occ.
FILTER(?occ!=wd:Q82955)
}
GROUP BY ?occ
}
#Labels
?occ <http://schema.org/name> ?name .
}
ORDER BY DESC(?howMany)
LIMIT 3
"""

print("Results")
run_query(queryString)

Results
[('occ', 'http://www.wikidata.org/entity/Q1749879'), ('name', 'pensioner'), ('howMany', '14823')]
[('occ', 'http://www.wikidata.org/entity/Q97768274'), ('name', 'anciens cadres'), ('howMany', '5440')]
[('occ', 'http://www.wikidata.org/entity/Q3062119'), ('name', 'farm operator'), ('howMany', '4907')]


3

In [4]:
#Q34
#count number of politicians in France
queryString = """
SELECT (COUNT(DISTINCT ?pol) AS ?howMany)
WHERE { 

?pol wdt:P106/wdt:P279* wd:Q82955;
     wdt:P27 wd:Q142.
}
LIMIT 3
"""

print("Results")
run_query(queryString)

Results
[('howMany', '81679')]


1

I am curious about "farm operator" on third spot, I want to check if there are some reasons.

In [6]:
#Q35
#result of previous query is odd, i want to see why "farm operator" is on third spot
queryString = """
SELECT ?pname (COUNT(DISTINCT ?pol) AS ?howMany)
WHERE { 

?pol wdt:P106/wdt:P279* wd:Q82955;
     wdt:P27 wd:Q142;
     wdt:P106 wd:Q3062119;
     wdt:P102 ?party.
?party <http://schema.org/name> ?pname .
}
GROUP BY ?pname
ORDER BY DESC(?howMany)
LIMIT 10
"""

print("Results")
run_query(queryString)


Results
[('pname', 'Union for a Popular Movement'), ('howMany', '10')]
[('pname', 'Rally for the Republic'), ('howMany', '5')]
[('pname', 'The Republicans'), ('howMany', '4')]
[('pname', 'National Rally'), ('howMany', '2')]
[('pname', 'La République En Marche'), ('howMany', '2')]
[('pname', 'Socialist Party'), ('howMany', '2')]
[('pname', 'Radical Party of the Left'), ('howMany', '2')]
[('pname', 'Radical Party'), ('howMany', '1')]
[('pname', 'French Communist Party'), ('howMany', '1')]
[('pname', 'Group Centrist Union'), ('howMany', '1')]


10

In [7]:
#Q36
#group by political party is not useful
#check if there are some common properties that can be useful
queryString = """
SELECT ?p (COUNT(?pol) AS ?howMany)
WHERE { 

?pol wdt:P106/wdt:P279* wd:Q82955;
     wdt:P27 wd:Q142;
     wdt:P106 wd:Q3062119;
     ?p ?o.
}
GROUP BY ?p
ORDER BY DESC(?howMany)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P39'), ('howMany', '14122')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('howMany', '12972')]
[('p', 'http://www.wikidata.org/prop/direct/P569'), ('howMany', '4951')]
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('howMany', '4908')]
[('p', 'http://www.wikidata.org/prop/direct/P735'), ('howMany', '4908')]
[('p', 'http://www.wikidata.org/prop/direct/P27'), ('howMany', '4907')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('howMany', '4907')]
[('p', 'http://www.wikidata.org/prop/direct/P21'), ('howMany', '4907')]
[('p', 'http://www.wikidata.org/prop/direct/P734'), ('howMany', '4887')]
[('p', 'http://schema.org/name'), ('howMany', '4854')]


10

In [8]:
#Q37
#P39 stands for "position held" (found in Q14)
queryString = """
SELECT ?pname (COUNT(DISTINCT ?pol) AS ?howMany)
WHERE { 

?pol wdt:P106/wdt:P279* wd:Q82955;
     wdt:P27 wd:Q142;
     wdt:P106 wd:Q3062119;
     wdt:P39 ?pos.
?pos <http://schema.org/name> ?pname .
}
GROUP BY ?pname
ORDER BY DESC(?howMany)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('pname', 'community councilor of Ternois'), ('howMany', '17')]
[('pname', 'Vals de Saintonge community counselor'), ('howMany', '15')]
[('pname', 'community counselor of Champagnole Nozeroy Jura'), ('howMany', '14')]
[('pname', 'president'), ('howMany', '13')]
[('pname', 'regional councilor of Nouvelle-Aquitaine'), ('howMany', '11')]
[('pname', 'member of the Senate of France'), ('howMany', '10')]
[('pname', 'conseiller régional du Grand Est'), ('howMany', '7')]
[('pname', 'member of the general council'), ('howMany', '7')]
[('pname', 'member of the French National Assembly'), ('howMany', '7')]
[('pname', 'conseiller régional des Hauts-de-France'), ('howMany', '7')]


10

<h3> Task 5.2 </h3>
What if you consider only politicians for which we don't have a date of death?

Looking for date of death property

In [12]:
#Q38
#looking for date of death property
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 

?pol wdt:P106/wdt:P279* wd:Q82955;
     ?p ?o.
?p <http://schema.org/name> ?pname .

FILTER REGEX(?pname, ".*death.*")

}
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1196'), ('pname', 'manner of death')]
[('p', 'http://www.wikidata.org/prop/direct/P20'), ('pname', 'place of death')]
[('p', 'http://www.wikidata.org/prop/direct/P509'), ('pname', 'cause of death')]
[('p', 'http://www.wikidata.org/prop/direct/P570'), ('pname', 'date of death')]


4

In [13]:
#Q39
#run query for task 5.2 using P570 found above
#retrieve top 3 occupations in Italy

queryString = """
SELECT ?occ ?name ?howMany WHERE {
{SELECT ?occ (COUNT(?pol) AS ?howMany)
WHERE { 

?pol wdt:P106/wdt:P279* wd:Q82955;
     wdt:P27 wd:Q38;
     wdt:P106 ?occ.
FILTER(?occ!=wd:Q82955)
FILTER NOT EXISTS{?pol wdt:P570 ?death.}
}
GROUP BY ?occ
}
#Labels
?occ <http://schema.org/name> ?name .
}
ORDER BY DESC(?howMany)
LIMIT 3
"""

print("Results")
run_query(queryString)

Results
[('occ', 'http://www.wikidata.org/entity/Q1930187'), ('name', 'journalist'), ('howMany', '333')]
[('occ', 'http://www.wikidata.org/entity/Q40348'), ('name', 'lawyer'), ('howMany', '250')]
[('occ', 'http://www.wikidata.org/entity/Q36180'), ('name', 'writer'), ('howMany', '137')]


3

In [14]:
#Q40
#run query for task 5.2 using P570 found above
#retrieve top 3 occupations in France

queryString = """
SELECT ?occ ?name ?howMany WHERE {
{SELECT ?occ (COUNT(?pol) AS ?howMany)
WHERE { 

?pol wdt:P106/wdt:P279* wd:Q82955;
     wdt:P27 wd:Q142;
     wdt:P106 ?occ.
FILTER(?occ!=wd:Q82955)
FILTER NOT EXISTS{?pol wdt:P570 ?death.}
}
GROUP BY ?occ
}
#Labels
?occ <http://schema.org/name> ?name .
}
ORDER BY DESC(?howMany)
LIMIT 3
"""

print("Results")
run_query(queryString)

Results
[('occ', 'http://www.wikidata.org/entity/Q1749879'), ('name', 'pensioner'), ('howMany', '14712')]
[('occ', 'http://www.wikidata.org/entity/Q97768274'), ('name', 'anciens cadres'), ('howMany', '5440')]
[('occ', 'http://www.wikidata.org/entity/Q3062119'), ('name', 'farm operator'), ('howMany', '4893')]


3

<h3> Task 5.3 </h3>
Which politicians had a spouse that was also a politician? How many in each country?

In [20]:
#Q41
#from Q14 I know that P26 = spouse
queryString = """
SELECT ?cname (COUNT(DISTINCT ?pol) AS ?howMany)
WHERE { 
?pol wdt:P106/wdt:P279 wd:Q82955;
     wdt:P26 ?s;
     wdt:P27 ?country.
?s wdt:P106/wdt:P279 wd:Q82955.
?country <http://schema.org/name> ?cname .
}
GROUP BY ?cname
ORDER BY DESC(?howMany)
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('cname', 'Soviet Union'), ('howMany', '4')]
[('cname', 'Ancient Egypt'), ('howMany', '3')]
[('cname', 'Ancient Rome'), ('howMany', '2')]
[('cname', 'Russian Empire'), ('howMany', '2')]
[('cname', 'Russia'), ('howMany', '2')]
[('cname', 'New Zealand'), ('howMany', '2')]
[('cname', 'Ukrainian Soviet Socialist Republic'), ('howMany', '1')]
[('cname', 'Canada'), ('howMany', '1')]
[('cname', 'Australia'), ('howMany', '1')]
[('cname', 'United States of America'), ('howMany', '1')]


10

In the previous query i had to delete the iteration over P279 in order to be able to execute it.

<h3> Task 5 ENDS HERE </h3>
The queries confirmed what i found out in the previous task, i.e. in Italy most of politicians are lawyer or journalist. Also I noted that adding
the date of birth the number of results went down but the top 3 was the same. About France, the results are different: the top 3 is pensioner,anciens cadres and farm operator. Even adding the date of birth, the results are the same. 